# Scraping Google Career Webpages

## Prerequisites
* selenium
* one of the following (depending on which browser you're using)
  * firefox: [geckodriver](https://github.com/mozilla/geckodriver/releases/)
  * chrome/chromium: [chromedriver](http://chromedriver.chromium.org/)
  
## Useful Tutorials
* https://huilansame.github.io/huilansame.github.io/archivers/sleep-implicitlywait-wait
* https://wangxin1248.github.io/python/2018/09/python3-spider-8.html

In [2]:
class GoogleJob:
    """ Wraps job title, location, minimum/preferred qualifications and responsibilities """
    def __init__(self, title, location, minimum_qual, preferred_qual, responsibilities):
        self.title = title
        self.location = location
        self.minimum_qual = minimum_qual
        self.preferred_qual = preferred_qual
        self.responsibilities = responsibilities
        
    def __repr__(self):
        return '{} @ {}'.format(self.title, self.location)

## 1. Scraping a single page into a GoogleJob object
Use `scrape_job()` provided below on single job with its url.

Example target: https://careers.google.com/jobs/results/6163626811654144-front-end-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance

In [53]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException
import selenium.webdriver.support.ui as ui

import pandas
import time
import csv

In [28]:
def _extract(class_name: str):
    """ Extracts the specified element by class name 
    :return: readable text in the element.
    """
    return driver.find_element_by_class_name(class_name).text

In [47]:
def scrape_job(url: str, wait: WebDriverWait):
    """ Scrape the job info from the specified Url. A broswer driver MUST be initialized beforehand.
    :param url: the url of a detailed google job page.
    :param wait: contains timeout.
    :return: a GoogleJob object.
    """
    driver.get(url)
    
    # Wait until all required elements are generated.
    try:
        wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'gc-card__title')))
        wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'gc-job-tags__location')))
        wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'gc-job-detail__section--qualifications')))
        wait.until(ec.presence_of_element_located((By.CLASS_NAME, 'gc-job-detail__section--responsibilities')))
    except TimeoutException:
        return None
    
    # Extract job information.
    title = _extract('gc-card__title')
    location = _extract('gc-job-tags__location')
    qualifications = _extract('gc-job-detail__section--qualifications').split('\n\n')
    minimum_qual = qualifications[0].replace('Minimum qualifications:\n', '')
    preferred_qual = qualifications[1].replace('Preferred qualifications:\n', '') if len(qualifications) > 1 else ''
    responsibilities = _extract('gc-job-detail__section--responsibilities').replace('Responsibilities\n', '', 1)
    
    return GoogleJob(title, location, minimum_qual, preferred_qual, responsibilities)

In [56]:
options = Options()
options.add_argument('-headless')
driver = Firefox(executable_path='/opt/firefox/geckodriver', options=options)

wait = WebDriverWait(driver, timeout=10)
job = scrape_job(r'https://careers.google.com/jobs/results/4552786006179840-software-engineer-android-framework/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=35&q=software&sort_by=relevance', wait)

if job is not None:
    print(vars(job))

driver.quit()

{'title': 'Software Engineer, Android Framework', 'location': 'Mountain View, CA, USA', 'minimum_qual': 'Qualifications\nBA/BS degree in Computer Science or related technical field, or equivalent practical experience.\n1 year of professional software development experience.\nExperience with the Java programming language and Android framework.', 'preferred_qual': 'Experience using Android UI Toolkit and/or framework APIs or equivalent on other platforms.\nExperience with system design on Android and/or other operating systems.\nInterest in application architecture and productivity libraries.', 'responsibilities': 'Help create compelling APIs that allow developers to create efficient, powerful and performant applications.\nWork on existing and new UI functionality and APIs, such as UI components, animations, text, layouts and the event processing system.\nWork on system-level framework functionality such as activity management, user profiles, storage, and optimizing system performance an

## Write the dict representation of a GoogleJob object to CSV

In [43]:
csv_file = 'google_jobs.csv'
job_dict = vars(job)

with open(csv_file, 'w') as f:
    w = csv.DictWriter(f, job_dict.keys())
    w.writeheader()
    w.writerow(job_dict)
    
print('File written: ' + csv_file)

File written: google_jobs.csv


## CSV to Pandas DataFrame

In [44]:
dat = pandas.read_csv(csv_file)
dat

,title,location,minimum_qual,preferred_qual,responsibilities
0,"Software Engineer, Android Framework","Mountain View, CA, USA",Qualifications\nBA/BS degree in Computer Scien...,Experience using Android UI Toolkit and/or fra...,Help create compelling APIs that allow develop...


---

## 2. Search & Scrape All Relevant Jobs
Use `scrape_jobs(keyword, wait)` provided below on all jobs relevant to a specific keyword.

Example: all jobs related to the keyword `software`.

In [49]:
def scrape_jobs(keyword: str, wait: WebDriverWait):
    """ Scrape info of all jobs related to the specified keyword
    :param keyword: google job search keyword.
    :param wait: contains timeout.
    :return: a list of GoogleJob objects.
    """
    # Open Google job search page.
    driver.get(r'https://careers.google.com/jobs/results/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q={}&sort_by=relevance'.format(keyword));
    
    # VERY DIRTY WORKAROUND :(
    # There's a weird bug. I cannot get url count from any of the search result pages.
    # However, I can get it from one of the job's detail page.
    wait.until(ec.presence_of_element_located((By.ID, 'search-results')))
    wait.until(ec.presence_of_element_located((By.XPATH, "//a[@data-gtm-ref='job-results-card']")))
    result_pane = driver.find_element_by_id('search-results')
    cards = result_pane.find_elements_by_xpath("//a[@data-gtm-ref='job-results-card']")
    driver.get(cards[0].get_attribute('href'))
    
    url_count_class_name = 'gc-jobs-matched__count--active'
    wait.until(ec.presence_of_element_located((By.CLASS_NAME, url_count_class_name)))
    url_count = int(driver.find_element_by_class_name(url_count_class_name).text)
    driver.back()
    
    urls_per_page = 20
    page_count = (url_count // urls_per_page) + 1
    
    # Loop until there's no `next` hyperlink.
    urls = []
    
    for i in range(0, page_count):
        try:
            time.sleep(2) # Sleep for 2 secs for the page to load or it will scream like a bitch
            
            wait.until(ec.presence_of_element_located((By.ID, 'search-results')))
            wait.until(ec.presence_of_element_located((By.XPATH, "//a[@data-gtm-ref='job-results-card']")))
            result_pane = driver.find_element_by_id('search-results')
            cards = result_pane.find_elements_by_xpath("//a[@data-gtm-ref='job-results-card']")
            
            urls += [card.get_attribute('href') for card in cards]
            print(len(urls))
        
            # If `next` cannot be found after `timeout` seconds, it will throw 
            # a TimeoutException, then we can break the loop.
            wait.until(ec.presence_of_element_located((By.XPATH, "//a[@data-gtm-ref='search-results-next-click']")))
            driver.find_element_by_xpath("//a[@data-gtm-ref='search-results-next-click']").send_keys(Keys.RETURN)
        except Exception as e:
            print(e)
            break
            
    print('url_done: ' + str(len(urls)))
    print(urls)
    
    # For each url in urls, scrape_job() and get a GoogleJob object,
    # hence this could take a while.
    jobs = []
    for i, url in enumerate(urls):
        print('\rProcessing ({}/{}): {}'.format(i, len(urls), url))
        job = scrape_job(url, wait)
        if job is not None:
            jobs.append(job)
            
    return jobs

In [50]:
options = Options()
options.add_argument('-headless')
driver = Firefox(executable_path='/opt/firefox/geckodriver', options=options)

wait = WebDriverWait(driver, timeout=10)
jobs = scrape_jobs('software', wait)
driver.quit()

print('Collected ' + str(len(jobs)) + ' jobs.')

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1713
url_done: 1713
['https://careers.google.com/jobs/results/6163626811654144-front-end-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance', 'https://careers.google.com/jobs/results/6343639863328768-software-engineer-html5-video-google-cloud-platform/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance', 'https://careers.google.com/jobs/results/5590805127561216-front-end-software-engineer-youtube/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relev

Processing (1/1713): https://careers.google.com/jobs/results/6343639863328768-software-engineer-html5-video-google-cloud-platform/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance
Processing (2/1713): https://careers.google.com/jobs/results/5590805127561216-front-end-software-engineer-youtube/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance
Processing (3/1713): https://careers.google.com/jobs/results/5175961349980160-software-engineer-google-home/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance
Processing (4/1713): https://careers.google.com/jobs/results/5629038607663104-software-engineer-front-end-development/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&q=software&sort_by=relevance
Processing (5/1713): https://careers.google.com/jobs/results/6489007460646912-wireless-software-engineer-g

Processing (38/1713): https://careers.google.com/jobs/results/6513897049358336-software-engineer-database-systems/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=2&q=software&sort_by=relevance
Processing (39/1713): https://careers.google.com/jobs/results/5104577030586368-technical-lead-gpu-system-software/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=2&q=software&sort_by=relevance
Processing (40/1713): https://careers.google.com/jobs/results/6734427092877312-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=3&q=software&sort_by=relevance
Processing (41/1713): https://careers.google.com/jobs/results/4893047012917248-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=3&q=software&sort_by=relevance
Processing (42/1713): https://careers.google.com/jobs/results/6022385536008192-software-engineer/?company=Google&company=YouTu

Processing (77/1713): https://careers.google.com/jobs/results/4949537905115136-full-stack-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=4&q=software&sort_by=relevance
Processing (78/1713): https://careers.google.com/jobs/results/5758278481149952-software-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=4&q=software&sort_by=relevance
Processing (79/1713): https://careers.google.com/jobs/results/4838776012734464-technical-solutions-engineer-gchips-software-power-and-performance/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=4&q=software&sort_by=relevance
Processing (80/1713): https://careers.google.com/jobs/results/5394659566354432-software-engineer-windows-internals-google-cloud/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=5&q=software&sort_by=relevance
Processing (81/1713): https://careers.google.com/jobs/results/5885667

Processing (113/1713): https://careers.google.com/jobs/results/6731630733623296-software-engineer-youtube-machine-learning/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=6&q=software&sort_by=relevance
Processing (114/1713): https://careers.google.com/jobs/results/6047583824969728-research-software-engineer-google-ai/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=6&q=software&sort_by=relevance
Processing (115/1713): https://careers.google.com/jobs/results/6020218331791360-data-center-network-deployment-lead-technical-program-management-hardware-operations/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=6&q=software&sort_by=relevance
Processing (116/1713): https://careers.google.com/jobs/results/6023283369771008-software-engineer-android-applications-verily-life-sciences-cambridge/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=6&q=software&sort_by=rel

...

Processing (1549/1713): https://careers.google.com/jobs/results/6343153659609088-engineering-director-actions-on-google/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=78&q=software&sort_by=relevance
Processing (1550/1713): https://careers.google.com/jobs/results/6473808502259712-business-development-manager-google-cloud-mandarin-english/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=78&q=software&sort_by=relevance
Processing (1551/1713): https://careers.google.com/jobs/results/6545908453801984-field-sales-representative-google-cloud-english-japanese/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=78&q=software&sort_by=relevance
Processing (1552/1713): https://careers.google.com/jobs/results/6416103843495936-vendor-program-manager-velocity/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=78&q=software&sort_by=relevance
Processing (1553/1713): https://c

Processing (1584/1713): https://careers.google.com/jobs/results/6740893047652352-technical-program-manager-site-reliability-engineering/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=80&q=software&sort_by=relevance
Processing (1585/1713): https://careers.google.com/jobs/results/6033539515744256-data-scientist-engineering/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=80&q=software&sort_by=relevance
Processing (1586/1713): https://careers.google.com/jobs/results/5125904159408128-technical-program-manager-voice-infrastructure-and-operations/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=80&q=software&sort_by=relevance
Processing (1587/1713): https://careers.google.com/jobs/results/4512603482292224-strategic-negotiator-google-cloud-deal-operations/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=80&q=software&sort_by=relevance
Processing (1588/1713): ht

Processing (1619/1713): https://careers.google.com/jobs/results/6198887032487936-technical-program-manager-site-reliability-engineering/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=81&q=software&sort_by=relevance
Processing (1620/1713): https://careers.google.com/jobs/results/5322482245632000-data-scientist-technical-lead-google-maps/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=82&q=software&sort_by=relevance
Processing (1621/1713): https://careers.google.com/jobs/results/5736158073126912-strategic-negotiator-google-cloud-deal-operations/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=82&q=software&sort_by=relevance
Processing (1622/1713): https://careers.google.com/jobs/results/6067878098042880-field-sales-manager-google-cloud/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=82&q=software&sort_by=relevance
Processing (1623/1713): https://careers.

Processing (1654/1713): https://careers.google.com/jobs/results/5633469411688448-director-site-reliability-engineering/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=83&q=software&sort_by=relevance
Processing (1655/1713): https://careers.google.com/jobs/results/4814606247854080-principal-strategy-and-insights-manager-gaming-and-apps/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=83&q=software&sort_by=relevance
Processing (1656/1713): https://careers.google.com/jobs/results/6196003319513088-program-manager-product-supportability-g-suite/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=83&q=software&sort_by=relevance
Processing (1657/1713): https://careers.google.com/jobs/results/4896134717964288-field-sales-representative-retail/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=83&q=software&sort_by=relevance
Processing (1658/1713): https://careers.googl

Processing (1689/1713): https://careers.google.com/jobs/results/6744242443714560-strategic-partnerships-manager-google-cloud-iot/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=85&q=software&sort_by=relevance
Processing (1690/1713): https://careers.google.com/jobs/results/5446146451505152-data-center-electrical-engineer/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=85&q=software&sort_by=relevance
Processing (1691/1713): https://careers.google.com/jobs/results/5886943927926784-head-of-regional-partner-marketing-google-cloud/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=85&q=software&sort_by=relevance
Processing (1692/1713): https://careers.google.com/jobs/results/4671261152313344-account-manager-search-ads-360/?company=Google&company=YouTube&employment_type=FULL_TIME&hl=en_US&jlo=en_US&page=85&q=software&sort_by=relevance
Processing (1693/1713): https://careers.google.com/jobs/resul

## Write the info of all jobs to CSV

In [51]:
csv_file = 'google_jobs.csv'

with open(csv_file, 'w') as f:
    w = csv.DictWriter(f, vars(jobs[0]).keys())
    w.writeheader()
    
    for job in jobs:
        w.writerow(vars(job))
    
print('File written: ' + csv_file)

File written: google_jobs.csv


In [52]:
dat = pandas.read_csv(csv_file)
dat

,title,location,minimum_qual,preferred_qual,responsibilities
0,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBA/BS degree or equivalent pra...,"4 years of relevant work experience, including...",Build next-generation web applications with a ...
1,"Software Engineer, HTML5 Video, Google Cloud P...","Sunnyvale, CA, USA",Qualifications\nBS degree in Electrical Engine...,MS degree in Electrical Engineering or Compute...,"Design, implement and launch complex HTML5 vid..."
2,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBA/BS in Computer Science or r...,Experience with one or more general purpose pr...,"Design, implement and launch highly-visible, p..."
3,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBachelor's degree in Computer ...,Experience working with hardware designers/rea...,Develop the whole software stack for consumer ...
4,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBachelor's degree in a technic...,"Development experience in designing modular, o...",Build next-generation web applications with a ...
5,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nMaster's degree in Electrical ...,PhD degree.\nExperience with wireless protocol...,"Participate in architecting, developing, testi..."
6,Front End Software Engineer,"Pittsburgh, PA, USA","Qualifications\nBS degree in Computer Science,...",10 years of relevant work experience in softwa...,Work alongside the Technical Lead to drive lon...
7,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBachelor's degree in Computer ...,"Master’s degree or PhD in Engineering, Compute...","Design, develop, test, deploy, maintain and im..."
8,"Software Engineer, Infrastructure (English)","Tel Aviv-Yafo, Israel",Qualifications\nBachelor's degree in Computer ...,Master’s degree.\nExperience with Unix/Linux o...,"Design, develop, test, deploy, maintain and im..."
9,Front End Software Engineer,"Pittsburgh, PA, USA",Qualifications\nBA/BS degree in Computer Scien...,"Experience in API design, performance optimiza...","Manage individual project priorities, deadline..."
